In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
5 % 3

2

In [77]:
from sts_curves import *
from columbus import compute_S_l0, computeA, STS, alg2, alg1
import numpy as np
from surface_dynamics.all import *
from tqdm import tqdm

In [3]:
a = {1: 2, 2:3, 3:1, 4:5, 5:6, 6:7, 7:8, 8:4}

b = {1: 2, 2:3, 3:1, 4:5, 5:6, 6:7, 7:8, 8:4, 9:9}

In [4]:
get_cycles(a)

min(*get_cycles(a),*get_cycles(b))

1

In [5]:
computeA((1,1))

[[2, -1], [-1, 1]]

In [6]:
compute_S_l0(3)

[(1, 0), (0, 1), (1, 1), (-1, 1), (1, 2), (-1, 2), (2, 1), (-2, 1)]

In [7]:
verify = np.array((1,0))

for v in compute_S_l0(100):
    A = computeA(v)
    A = np.array(A)
    if round(np.linalg.det(A)) != 1:
        print('bad det')
        print(np.linalg.det(A))
        break

    i = A @ np.array(v)
    if not np.array_equal(i, verify):
        print('bad A')

print('success')

success


In [9]:
def get_cycles(p: dict) -> list:
    '''
    returns permutation p
    as a string in cycle notation
    '''

    n = len(p)

    visited = set()
    cycles = []
    perm = ''

    for i in range(1,n+1):
        if i in visited:
            continue

        j = i
        cycle = []
        while j not in visited:
            visited.add(j)
            cycle.append(str(j))
            j = p[j]
        
        # print(cycle)
        cycles.append(cycle)

    return cycles

In [10]:
a = {1:2, 2:3, 3:1, 4:4, 5:6, 6:7, 7:8, 8:5}

In [11]:
get_cycles(a)

[['1', '2', '3'], ['4'], ['5', '6', '7', '8']]

Example from Columbus

In [104]:
h = {13:14, 14:15, 15:13, 7:8, 8:9, 9:10, 10:11, 11:12, 12:7, 1:2, 2:3, 3:4, 4:5, 5:6, 6:1}
v= {1:7, 7:13, 13:1, 2:8, 8:14, 14:2, 3:9, 9:15, 15:3, 4:10, 10:6, 6:12, 12:5, 5:11, 11:4} 

In [13]:
ex = STS(h,v)

In [14]:
alg2(ex, (1,1))

{1, 2, 10, 14}


{(6, 13): [((1, 4), 7.0710678118654755), ((6, 13), 4.242640687119286)],
 (1, 4): [((1, 4), 4.242640687119286), ((6, 13), 5.656854249492381)]}

In [19]:
g = alg1(ex)

# testing shortest path

In [42]:
g[(1,4)]

{((1, 4), 2.23606797749979),
 ((1, 4), 2.8284271247461903),
 ((1, 4), 3.0),
 ((1, 4), 4.242640687119286),
 ((1, 4), 4.47213595499958),
 ((1, 4), 6.708203932499369),
 ((6, 13), 2.0),
 ((6, 13), 4.47213595499958),
 ((6, 13), 5.656854249492381),
 ((6, 13), 8.94427190999916)}

In [57]:
import heapq
# thanks ChatGPT for this code

def shortest_closed_path(graph):
    # Find the shortest path from each vertex to every other vertex
    shortest_paths = {}
    for start_vertex in graph:
        dists = {vertex: float('inf') for vertex in graph}
        dists[start_vertex] = 0

        queue = [(0, start_vertex)]
        while queue:
            (distance, current_vertex) = heapq.heappop(queue)

            # Shortest path to current_vertex found
            if current_vertex in shortest_paths:
                continue

            shortest_paths[current_vertex] = distance
            for neighbor, weight in graph[current_vertex]:
                if neighbor not in shortest_paths:
                    heapq.heappush(queue, (distance + weight, neighbor))

    # Find the shortest closed path (cycle) in the graph
    shortest_cycle = float('inf')
    for start_vertex in graph:
        for neighbor, weight in graph[start_vertex]:
            cycle_distance = shortest_paths[start_vertex] + weight + shortest_paths[neighbor]
            if cycle_distance < shortest_cycle:
                shortest_cycle = cycle_distance

    return shortest_cycle

In [58]:
shortest_closed_path(g)    

2.23606797749979

In [56]:
n

defaultdict(<function dijkstra.<locals>.<lambda> at 0x141e3ede0>, {(1, 4): 0, (6, 13): 2.0})

In [49]:
g

{(1, 4): {((1, 4), 2.23606797749979),
  ((1, 4), 2.8284271247461903),
  ((1, 4), 3.0),
  ((1, 4), 4.242640687119286),
  ((1, 4), 4.47213595499958),
  ((1, 4), 6.708203932499369),
  ((6, 13), 2.0),
  ((6, 13), 4.47213595499958),
  ((6, 13), 5.656854249492381),
  ((6, 13), 8.94427190999916)},
 (6, 13): {((1, 4), 1.0),
  ((1, 4), 2.23606797749979),
  ((1, 4), 4.0),
  ((1, 4), 7.0710678118654755),
  ((1, 4), 11.180339887498949),
  ((6, 13), 2.23606797749979),
  ((6, 13), 2.8284271247461903),
  ((6, 13), 3.0),
  ((6, 13), 4.242640687119286),
  ((6, 13), 4.47213595499958),
  ((6, 13), 6.708203932499369)}}

In [20]:
visited = set{}
shortest_closed_path = float('inf')

for node in g.keys():
    if node in visited:
        continue
    

    min_path, visited = shortest_closed_path(node, g, visited)
    shortest_closed_path = min(min_path, shortest_closed_path)


dict_keys([(1, 4), (6, 13)])

In [21]:
g[(1,4)]

{((1, 4), 2.23606797749979),
 ((1, 4), 2.8284271247461903),
 ((1, 4), 3.0),
 ((1, 4), 4.242640687119286),
 ((1, 4), 4.47213595499958),
 ((1, 4), 6.708203932499369),
 ((6, 13), 2.0),
 ((6, 13), 4.47213595499958),
 ((6, 13), 5.656854249492381),
 ((6, 13), 8.94427190999916)}

In [26]:
component = AbelianStratum(1, 1).hyperelliptic_component()
curves = component.arithmetic_teichmueller_curves(10)


In [61]:
curves[0].origami().u()

(1,2,3,4,5,6,7)(8,9,10)

In [38]:
curves[0].origami().r()

(7,8)

In [63]:
import re
strs = '(2,3)(4,5,6)'
regex = r'\(.+?\)|".+?"|\w+' 
re.findall(regex, strs)

['(2,3)', '(4,5,6)']

In [76]:
curves[0].origami().r().cycle_string()

'(7,8)'

In [75]:
STS.get_cycle_dict(curves[0].origami().r().cycle_string(), 10)

{7: 8, 8: 7, 1: 1, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6, 9: 9, 10: 10}

In [84]:
curves

[Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7,8)(9)(10)
 (1,2,3,4,5,6,7)(8,9,10),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8,9,10)
 (1,2,3,4,5,6,7,8)(9,10),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8,9)(10)
 (1,2,3,4,5,6,7,8)(9,10),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5,6)(7)(8)(9)(10)
 (1,2,3,4,5)(6,7,8,9,10),
 Teichmueller curve of the origami
 (1)(2)(3)(4,5)(6,7)(8)(9)(10)
 (1,2,3,4,6)(5,7,8,9,10)]

In [97]:
curve.origami().stratum()

H_2(1^2)

In [106]:
curves

[Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9)(10)(11)(12)(13,14)(15)
 (1,2,3,4,5,6,7,8,9,10,11,12,13)(14,15),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9)(10)(11)(12,13)(14)(15)
 (1,2,3,4,5,6,7,8,9,10,11,12)(13,14,15),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9)(10)(11,12,13,14,15)
 (1,2,3,4,5,6,7,8,9,10,11)(12,14)(13,15),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9)(10)(11,12)(13,14)(15)
 (1,2,3,4,5,6,7,8,9,10,11,13,15,12,14),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9)(10,11)(12)(13)(14)(15)
 (1,2,3,4,5,6,7,8,9,10)(11,12,13,14,15),
 Teichmueller curve of the origami
 (1)(2)(3)(4)(5)(6)(7)(8)(9,10)(11,12)(13)(14)(15)
 (1,2,3,4,5,6,7,8,9,11,13,14,15,10,12)]

In [115]:
_, _, s_action = curve.origami().sl2z_edges()

i, s_orbit_reps = 0, list(s_action.keys())
while i < len(s_orbit_reps):
    o = s_orbit_reps[i]
    s_o = s_action[o]
    s_orbit_reps.remove(s_o)
    s_o = s_action[s_o]
    if s_o != o:
        s_orbit_reps.remove(s_o)
        s_o = s_action[s_o]
        s_orbit_reps.remove(s_o)
    i += 1

In [119]:
s_orbit_reps[0].r()

(1,2,3,4,5,6,7,8,9,10,11,12,13,14,15)

In [125]:
n = 25

component = AbelianStratum(1, 1).hyperelliptic_component()
curves = component.arithmetic_teichmueller_curves(n)

max_curve = 0

for curve in tqdm(curves):

    ####### Columbus Magic ##################
    # basically to get all the representatives of orbit under S
    _, _, s_action = curve.origami().sl2z_edges()

    i, s_orbit_reps = 0, list(s_action.keys())
    while i < len(s_orbit_reps):
        o = s_orbit_reps[i]
        s_o = s_action[o]
        s_orbit_reps.remove(s_o)
        s_o = s_action[s_o]
        if s_o != o:
            s_orbit_reps.remove(s_o)
            s_o = s_action[s_o]
            s_orbit_reps.remove(s_o)
        i += 1
    #######################################

    for o in s_orbit_reps:
        h = STS.get_cycle_dict(o.r().cycle_string(), n)
        v = STS.get_cycle_dict(o.u().cycle_string(), n)

        s = STS(h,v)

        g = alg1(s)

        sys_s = shortest_closed_path(g)

        max_curve = max(max_curve, sys_s)

max_curve

100%|██████████| 4/4 [03:17<00:00, 49.41s/it]


3.605551275463989